In [1]:
import pyomo.environ as pyomo
import pandas as pd

In [2]:
m = pyomo.ConcreteModel()

In [3]:
load=pd.read_csv('demand.csv')
solar=pd.read_csv('solar.csv')

In [4]:
T = 24
price_grid = 5
price_renew=2
deg_cost=10
soemax=40
soemin=0
pchmax=10
pchmin=0

In [5]:
S = solar['RealPower'].values
L = load['RealPower'].values

In [6]:
times = range(T)
tplus = range(T+1)

In [7]:
m.Pgrid = pyomo.Var(times, domain=pyomo.NonNegativeReals)
m.Pexport = pyomo.Var(times, domain=pyomo.NonNegativeReals)
m.soe=pyomo.Var(tplus, domain=pyomo.NonNegativeReals)
m.pch=pyomo.Var(times, domain=pyomo.Reals)
m.pdch=pyomo.Var(times, domain=pyomo.Reals)
m.deg=pyomo.Var(times, domain=pyomo.NonNegativeReals)

In [8]:
# objective
cost = sum(price_grid*m.Pgrid[t]
            - m.Pexport[t]*price_grid
            - price_grid*m.pdch[t]
            + price_grid*m.pch[t] 
            + deg_cost*m.deg[t] for t in times)

In [9]:
cost

In [10]:
m.cost = pyomo.Objective(expr = cost, sense=pyomo.minimize)

In [11]:
# constraints
m.cons = pyomo.ConstraintList()

In [12]:
for t in times:
    m.cons.add(m.pch[t]<=pchmax)
    m.cons.add(0<=m.pch[t])
    m.cons.add(m.pdch[t]<=pchmax)
    m.cons.add(0<=m.pdch[t])
    m.cons.add(m.soe[t]<=soemax)
    m.cons.add(0<=m.soe[t])
    m.cons.add(L[t] + m.Pexport[t] + m.pch[t] == m.Pgrid[t]+S[t]+m.pdch[t]+m.deg[t])
#     m.cons.add(m.Pgrid[t]*m.Pexport[t]==0)
    m.cons.add(m.Pexport[t]>=0)
    m.cons.add(m.Pgrid[t]>=0)
    m.cons.add(m.soe[t+1] == m.soe[t] + m.pch[t] - m.pdch[t])

m.cons.add(m.soe[0] == 10)
m.cons.add(m.Pgrid[10]==0)
m.cons.add(m.Pgrid[11]==0)
m.cons.add(m.Pgrid[12]==0)


In [13]:
print("**************")
print("gridprice",price_grid)
print("***************")
print("renewable price",price_renew)
print("***************")
print("solar generation",S)
print("**************")
print("load",L)
print("***************")


**************
gridprice 5
***************
renewable price 2
***************
solar generation [ 0.     0.     0.     0.     0.     0.     0.     0.106  7.924 19.317
 29.399 44.717 47.049 47.029 44.719 37.46  20.366  8.34   0.266  0.
  0.     0.     0.     0.   ]
**************
load [ 61.077  64.231  64.612  67.146  66.027  71.29   88.297 100.229  99.979
 101.697 105.989 118.934 124.29  121.083 129.931   0.      0.      0.
 126.02  125.546 132.675 126.564 126.309 122.349]
***************


In [14]:
solver = pyomo.SolverFactory("gurobi", solver_io="python")
solver.options["NonConvex"] = 2
solver.solve(m)

{'Problem': [{'Name': 'unknown', 'Lower bound': 10078.155, 'Upper bound': 10078.155, 'Number of objectives': 1, 'Number of constraints': 244, 'Number of variables': 145, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 145, 'Number of nonzeros': 412, 'Sense': 1, 'Number of solutions': 1}], 'Solver': [{'Name': 'Gurobi 9.50', 'Status': 'ok', 'Wallclock time': 0.013181686401367188, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.'}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [15]:
for c in m.cons:
    print(m.cons[c].lslack(), m.cons[c].uslack())

inf 10.0
0.0 inf
inf 0.0
10.0 inf
inf 30.0
10.0 inf
0.0 0.0
0.0 inf
51.077 inf
0.0 0.0
inf 10.0
0.0 inf
inf 10.0
0.0 inf
inf 40.0
0.0 inf
0.0 0.0
0.0 inf
64.231 inf
0.0 0.0
inf 10.0
0.0 inf
inf 10.0
0.0 inf
inf 40.0
0.0 inf
0.0 0.0
0.0 inf
64.612 inf
0.0 0.0
inf 10.0
0.0 inf
inf 10.0
0.0 inf
inf 40.0
0.0 inf
0.0 0.0
0.0 inf
67.146 inf
0.0 0.0
inf 10.0
0.0 inf
inf 10.0
0.0 inf
inf 40.0
0.0 inf
0.0 0.0
0.0 inf
66.027 inf
0.0 0.0
inf 10.0
0.0 inf
inf 10.0
0.0 inf
inf 40.0
0.0 inf
0.0 0.0
0.0 inf
71.29 inf
0.0 0.0
inf 0.0
10.0 inf
inf 10.0
0.0 inf
inf 40.0
0.0 inf
0.0 0.0
0.0 inf
98.297 inf
0.0 0.0
inf 0.0
10.0 inf
inf 10.0
0.0 inf
inf 30.0
10.0 inf
0.0 0.0
0.0 inf
110.123 inf
0.0 0.0
inf 0.0
10.0 inf
inf 10.0
0.0 inf
inf 20.0
20.0 inf
0.0 0.0
0.0 inf
102.05499999999999 inf
0.0 0.0
inf 0.0
10.0 inf
inf 10.0
0.0 inf
inf 10.0
30.0 inf
0.0 0.0
0.0 inf
92.38 inf
0.0 0.0
inf 10.0
0.0 inf
inf 0.0
10.0 inf
inf 0.0
40.0 inf
0.0 0.0
0.0 inf
0.0 inf
0.0 0.0
inf 10.0
0.0 inf
inf 0.0
10.0 inf
inf 10.0

In [16]:
m.display()

Model unknown

  Variables:
    Pgrid : Size=24, Index=Pgrid_index
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     0 :             51.077 :  None : False : False : NonNegativeReals
          1 :     0 :             64.231 :  None : False : False : NonNegativeReals
          2 :     0 :             64.612 :  None : False : False : NonNegativeReals
          3 :     0 :             67.146 :  None : False : False : NonNegativeReals
          4 :     0 :             66.027 :  None : False : False : NonNegativeReals
          5 :     0 :              71.29 :  None : False : False : NonNegativeReals
          6 :     0 :             98.297 :  None : False : False : NonNegativeReals
          7 :     0 :            110.123 :  None : False : False : NonNegativeReals
          8 :     0 : 102.05499999999999 :  None : False : False : NonNegativeReals
          9 :     0 :              92.38 :  None : False : False : NonNegativeReals
         10 :     0

In [17]:
print("Total cost =", m.cost(), ".")
print("time   pgrid   pexport     SOE      CHARGE     DISCH    LOAD        pv        Deg")


cols = ['TIME',
        'PGRID',
        'PEXPORT',
        'SOE',
        'CHARGE',
        'DISCH',
        'LOAD',
        'PV',     
        'DEG']

df = pd.DataFrame(columns=cols)
for t in times:
    df = df.append({
        'TIME': t,
        'PGRID': pyomo.value(m.Pgrid[t]),
        'PEXPORT': pyomo.value(m.Pexport[t]),
        'SOE': pyomo.value(m.soe[t]),
        'CHARGE': pyomo.value(m.pch[t]),
        'DISCH': pyomo.value(m.pdch[t]),
        'LOAD': L[t],
        'PV': S[t],
        'DEG': pyomo.value(m.deg[t])
    },ignore_index=True)
df

Total cost = 10078.155 .
time   pgrid   pexport     SOE      CHARGE     DISCH    LOAD        pv        Deg


,TIME,PGRID,PEXPORT,SOE,CHARGE,DISCH,LOAD,PV,DEG
0,0.0,51.077,0.000,10.0,0.0,10.0,61.077,0.000,0.000
1,1.0,64.231,0.000,0.0,0.0,0.0,64.231,0.000,0.000
2,2.0,64.612,0.000,0.0,0.0,0.0,64.612,0.000,0.000
3,3.0,67.146,0.000,0.0,0.0,0.0,67.146,0.000,0.000
4,4.0,66.027,0.000,0.0,0.0,0.0,66.027,0.000,0.000
5,5.0,71.290,0.000,0.0,0.0,0.0,71.290,0.000,0.000
6,6.0,98.297,0.000,0.0,10.0,0.0,88.297,0.000,0.000
7,7.0,110.123,0.000,10.0,10.0,0.0,100.229,0.106,0.000
8,8.0,102.055,0.000,20.0,10.0,0.0,99.979,7.924,0.000
9,9.0,92.380,0.000,30.0,10.0,0.0,101.697,19.317,0.000


In [20]:
df.to_excel('soln1.xlsx')